# Fully Connected Neural Network on Iris Dataset

In this example, we show how one can train a neural network on a specific task (here Iris Classification) and use Concrete Framework to make the model work in FHE settings.

In [1]:
import torch
from torch import nn
import numpy as np

# Set the random seed for reproducibility
torch.manual_seed(1)

## Define our neural network

In [2]:
class FCIris(torch.nn.Module):
    """Neural network for Iris classification
    
    We define a fully connected network with 5 fully connected (fc) layers that 
    perform feature extraction and one (fc) layer to produce the final classification. 
    We will use 15 neurons on all the feature extractor layers to ensure that the FHE accumulators
    do not overflow (we are only allowed a maximum of 7 bits-width).

    Due to accumulator limits we have to design a deep network with few neurons on each layer. 
    This is in contrast to a traditional approach where the number of neurons increases after 
    each layer or block.
    """

    def __init__(self, input_size):
        super().__init__()

        # The first layer processes the input data, in our case 4 dimensional vectors 
        self.linear1 = nn.Linear(input_size, 15)
        self.sigmoid1 = nn.Sigmoid()
        # Next, we add four intermediate layers to perform features extraction
        self.linear2 = nn.Linear(15, 15)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(15, 15)
        self.sigmoid3 = nn.Sigmoid()
        self.linear4 = nn.Linear(15, 15)
        self.sigmoid4 = nn.Sigmoid()
        self.linear5 = nn.Linear(15, 15)
        self.sigmoid5 = nn.Sigmoid()
        # Finally, we add the decision layer for 3 output classes encoded as one-hot vectors
        self.decision = nn.Linear(15, 3)

    def forward(self, x):

        x = self.linear1(x)
        x = self.sigmoid1(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        x = self.linear3(x)
        x = self.sigmoid3(x)
        x = self.linear4(x)
        x = self.sigmoid4(x)
        x = self.linear5(x)
        x = self.sigmoid5(x)
        x = self.decision(x)

        return x


## Define all required variables to train the model

In [3]:
# Get iris dataset
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

# Split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train).float()
X_test = torch.tensor(X_test).float()
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

# Initialize our model
model = FCIris(X.shape[1])

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define the number of iterations
n_iters = 5000

# Define the batch size
batch_size = 16


## Train the model

In [4]:
for iter in range(n_iters):
        # Get a random batch of training data
        idx = torch.randperm(X_train.size()[0])
        X_batch = X_train[idx][:batch_size]
        y_batch = y_train[idx][:batch_size]
    
        # Forward pass
        y_pred = model(X_batch)
    
        # Compute loss
        loss = criterion(y_pred, y_batch)
    
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
    
        # Update weights
        optimizer.step()
    
        
        if iter % 1000 == 0:
            # Print epoch number, loss and accuracy
            print(f'Iterations: {iter:02} | Loss: {loss.item():.4f} | Accuracy: {100*(y_pred.argmax(1) == y_batch).float().mean():.2f}%')

Iterations: 00 | Loss: 1.1348 | Accuracy: 31.25%
Iterations: 1000 | Loss: 0.4863 | Accuracy: 68.75%
Iterations: 2000 | Loss: 0.0661 | Accuracy: 100.00%
Iterations: 3000 | Loss: 0.0185 | Accuracy: 100.00%
Iterations: 4000 | Loss: 0.0477 | Accuracy: 100.00%


## Predict with the torch model in clear

In [5]:
y_pred = model(X_test)

## Compile the model

In [6]:
from concrete.torch.compile import compile_torch_model
quantized_compiled_module = compile_torch_model(
    model,
    X_train,
    n_bits=2,
)

## Predict with the quantized model

In [7]:
# We now have a module in full numpy.
# Convert data to a numpy array.
X_train_numpy = X_train.numpy()
X_test_numpy = X_test.numpy()
y_train_numpy = y_train.numpy()
y_test_numpy = y_test.numpy()

quant_model_predictions = quantized_compiled_module(X_test_numpy)

## Predict in FHE

In [8]:
from tqdm import tqdm
homomorphic_quant_predictions = []
for x_q in tqdm(X_test_numpy):
    homomorphic_quant_predictions.append(
        quantized_compiled_module.forward_fhe.run(np.array([x_q]).astype(np.uint8))
    )
homomorphic_predictions = quantized_compiled_module.dequantize_output(
    np.array(homomorphic_quant_predictions, dtype=np.float32).reshape(quant_model_predictions.shape)
)

100%|██████████| 23/23 [04:14<00:00, 11.07s/it]


## Print the accuracy of both models

In [9]:
print(f'Test Accuracy: {100*(y_pred.argmax(1) == y_test).float().mean():.2f}%')
print(f'Test Accuracy Quantized Inference: {100*(quant_model_predictions.argmax(1) == y_test_numpy).mean():.2f}%')
print(f'Test Accuracy Homomorphic Inference: {100*(homomorphic_predictions.argmax(1) == y_test_numpy).mean():.2f}%') 


Test Accuracy: 100.00%
Test Accuracy Quantized Inference: 73.91%
Test Accuracy Homomorphic Inference: 73.91%


## Summary

In this notebook we presented a few steps to have a model (torch neural network) inference in over homomorphically encrypted data: 
- We first trained a fully connected neural network yielding ~100% accuracy
- Then quantized it using Concrete Framework. As we can see, the extreme post training quantization (only 2 bits of precision for weights, inputs and activations) made the neural network accruracy slighlty drop (~73%).
- We then use the compiled inference into its FHE equivalent to get our FHE predictions over the test set

The Homomorphic inference achieves a similar accuracy as the quantized model inference. 